In [1]:
import pandas as pd
import os
from collections import defaultdict
import paramiko # for ssh
import traceback #  for error handling
import ast
import requests
import re
from Bio.SeqUtils import seq1

c:\Users\liyoa\anaconda3\envs\sbna\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
cgc_data = pd.read_csv('Census_allTue Apr  9 04_58_38 2024.csv')
cgc_data.head()

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,10q11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG..."
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,10p12.1,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00..."
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,9q34.12,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007..."
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,1q25.2,yes,NaN,AML,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"ABLL,ARG,CCDS30947.1,ENSG00000143322.19,NM_007..."
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,2q37.3,yes,NaN,lipoma,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"CCDS2516.1,CMKOR1,CXCR7,ENSG00000144476.5,GPR1..."


In [21]:
# response = requests.get("https://gtexportal.org/api/v2/expression/geneExpression/gene-expression?gencodeId=ENSG00000065613.9,ENSG00000203782.5")
# response

In [2]:
def get_pdb_ids(gene_symbol, entrez_gene_id, synonyms):
    api_url = f"http://mygene.info/v3/query?q={gene_symbol}&species=human&fields=entrezgene,ensembl,pdb"
    response = requests.get(api_url)
    # check response
    if response.status_code != 200:
        return response.status_code, None
    response = response.json()
    for hit in response['hits']:
        if ( str(hit['_id']) == str(int(entrez_gene_id)) ) or ( synonyms and 'ensembl' in hit and hit['ensembl']['gene'] in synonyms ): # (sometimes entrez_gene_id might not match but ensembl gene does
            if 'pdb' in hit:
                return 200, hit['pdb'] # status_code, pdb_ids
            else: # no structures found 
                continue
    return 200, None


In [14]:
# add pdb structures to cgc_data
pdb_structures = []
count = 1
for _, row in cgc_data.iterrows():
    # print curent count, replace previous line    
    gene_symbol = row['Gene Symbol']
    entrez_gene_id = row['Entrez GeneId']
    synonyms = row['Synonyms']
    print(f"Processing {count} of {len(cgc_data)} ({gene_symbol})", end='\r')
    count += 1

    try:
        status, pdb = get_pdb_ids(gene_symbol, entrez_gene_id, synonyms)
    except Exception as e:
        print(f"Failed to get pdb ids for {gene_symbol} with error {e}")
        pdb_structures.append(None)
        continue

    if not pdb:
        print(f"Failed to get pdb ids for {gene_symbol} with status code {status}")
        pdb_structures.append(None)
    else:
        if type(pdb) == list:
            pdb_structures.append(pdb)
        else:
            pdb_structures.append([pdb])

cgc_data['PDB Structures'] = pdb_structures

Failed to get pdb ids for ACSL3 with status code 200
Failed to get pdb ids for ACSL6 with status code 200
Failed to get pdb ids for AFF3 with status code 200
Failed to get pdb ids for AKAP9 with status code 200
Failed to get pdb ids for ARHGEF10 with status code 200
Failed to get pdb ids for ARHGEF10L with status code 200
Failed to get pdb ids for ASPM with status code 200
Failed to get pdb ids for ASXL2 with status code 200
Failed to get pdb ids for ATF1 with status code 200
Failed to get pdb ids for ATP2B3 with status code 200
Failed to get pdb ids for BCL11B with status code 200
Failed to get pdb ids for BCL2L12 with status code 200
Failed to get pdb ids for BCL7A with status code 200
Failed to get pdb ids for BMP5 with status code 200
Failed to get pdb ids for BTG1 with status code 200
Failed to get pdb ids for CARS with error list indices must be integers or slices, not str
Failed to get pdb ids for CBFA2T3 with status code 200
Failed to get pdb ids for CCDC6 with status code 200


In [15]:
cgc_data.head(3)

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),...,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,PDB Structures,n_structures
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,10q11.23,yes,NaN,melanoma,...,E,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG...",[2CPD],4
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,10p12.1,yes,NaN,AML,...,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00...",[7LXE],4
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,9q34.12,yes,NaN,"CML, ALL, T-ALL",...,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007...","[1AB2, 1AWO, 1BBZ, 1JU5, 1OPL, 1ZZP, 2ABL, 2E2...",81


In [3]:
def dwl_pdb_file(pdb_id):
    base_url = 'https://files.rcsb.org/download/'
    pdb_url = base_url + pdb_id + '.pdb'
    response = requests.get(pdb_url)
    # Check if the request was successful
    if response.status_code == 200:
        # Save the PDB file to pdb_files folder
        with open(f'pdb_files/{pdb_id}.pdb', 'wb') as f:
            f.write(response.content)
    else:
        print(f"Failed to download PDB file {pdb_id}. Status code:", response.status_code)
    return

def get_resolution(pdb_id):
    try:
        with open(f'pdb_files/{pdb_id}.pdb', 'r') as f:
            pdb_text = f.read()

            # find line REMARK   2 RESOLUTION.
            resolution = None
            for line in pdb_text.split('\n'):
                if line.startswith('REMARK   2 RESOLUTION.'):
                    resolution = line.split("REMARK   2 RESOLUTION.")[1].strip()
                    break
        if "ANGSTROM" in resolution:
            resolution = resolution.replace(" ANGSTROMS.", "").strip()
        else:
            # print(f"Resolution not found for {pdb_id}")
            resolution = None
    except Exception as e:
        print(f"Failed to get resolution for {pdb_id} with error {e}")
        resolution = None
    return resolution


In [16]:
# count number of elements in each pdb_structure
lens = []
for i in pdb_structures:
    # if value is list
    if isinstance(i, list):
        lens.append(len(i))
    elif i is not None:
        lens.append(1)
    else:
        lens.append(0)

cgc_data['n_structures'] = lens
cgc_data.head(3)

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),...,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,PDB Structures,n_structures
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,10q11.23,yes,NaN,melanoma,...,E,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG...",[2CPD],1
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,10p12.1,yes,NaN,AML,...,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00...",[7LXE],1
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,9q34.12,yes,NaN,"CML, ALL, T-ALL",...,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007...","[1AB2, 1AWO, 1BBZ, 1JU5, 1OPL, 1ZZP, 2ABL, 2E2...",81


In [17]:
cgc_data.to_csv('Census_all_with_pdb.csv', index=False)

In [9]:
tmp_genes = ["KRAS", "EGFR", "TP53", "HRAS"]

In [10]:
# download pdb files

resolutions = defaultdict(dict)

# count number of structures for all gene
n_total = cgc_data[cgc_data['Gene Symbol'].isin(tmp_genes)]["n_structures"].sum()

for gene in tmp_genes:
    pdb_structures = cgc_data[cgc_data['Gene Symbol'] == gene]['PDB Structures'].values[0]
    if pdb_structures:
        for pdb_id in pdb_structures:
            n_total -= 1
            print(f"Downloading PDB file {pdb_id} for {gene} ({n_total} structures remaining)", end='\r')
            # check if the file already exists
            if not os.path.exists(f'pdb_files/{pdb_id}.pdb'):
                dwl_pdb_file(pdb_id)
            resolution = get_resolution(pdb_id)
            resolutions[gene][pdb_id] = resolution

Failed to download PDB file 7Q9U. Status code: 404 remaining))
Failed to get resolution for 7Q9U with error [Errno 2] No such file or directory: 'pdb_files/7Q9U.pdb'


In [18]:
def sort_pdb_ids_by_resolution(gene_id):
    pdb_resolutions = resolutions[gene_id]
    sorted_pdb_ids = sorted(pdb_resolutions, key=lambda pdb_id: float('inf') if pdb_resolutions[pdb_id] is None else float(pdb_resolutions[pdb_id]))
    sorted_resolutions = [(pdb_id, pdb_resolutions[pdb_id]) for pdb_id in sorted_pdb_ids]
    return sorted_resolutions

top_structures = dict()
n = 30
for gene_id in resolutions:
    sorted_pdb_resolutions = sort_pdb_ids_by_resolution(gene_id)
    print(f"\nTop {n} structures for Gene {gene_id}:")
    for pdb_id, resolution in sorted_pdb_resolutions[:n]:
        print(f"PDB ID: {pdb_id}, Resolution: {resolution}")
    top_structures[gene_id] = [i[0] for i in sorted_pdb_resolutions[:n]]


Top 30 structures for Gene KRAS:
PDB ID: 6P0Z, Resolution: 1.01
PDB ID: 8ONV, Resolution: 1.01
PDB ID: 8AZZ, Resolution: 1.02
PDB ID: 4QL3, Resolution: 1.04
PDB ID: 8AZX, Resolution: 1.04
PDB ID: 8B00, Resolution: 1.04
PDB ID: 8AZV, Resolution: 1.05
PDB ID: 8AZY, Resolution: 1.09
PDB ID: 8B78, Resolution: 1.11
PDB ID: 8AFB, Resolution: 1.12
PDB ID: 4TQA, Resolution: 1.13
PDB ID: 4LDJ, Resolution: 1.15
PDB ID: 6TAN, Resolution: 1.16
PDB ID: 7O70, Resolution: 1.18
PDB ID: 8EDY, Resolution: 1.18
PDB ID: 8EER, Resolution: 1.18
PDB ID: 4Q03, Resolution: 1.20
PDB ID: 7R0N, Resolution: 1.20
PDB ID: 8EBZ, Resolution: 1.20
PDB ID: 8TXH, Resolution: 1.20
PDB ID: 4OBE, Resolution: 1.24
PDB ID: 6QUW, Resolution: 1.24
PDB ID: 5XCO, Resolution: 1.25
PDB ID: 7RT1, Resolution: 1.27
PDB ID: 7T47, Resolution: 1.27
PDB ID: 4LUC, Resolution: 1.29
PDB ID: 4Q01, Resolution: 1.29
PDB ID: 6B0V, Resolution: 1.29
PDB ID: 7RT5, Resolution: 1.29
PDB ID: 7RPZ, Resolution: 1.30

Top 30 structures for Gene EGFR:
PD

In [36]:
# save top_structures to a file
with open(f'top_{n}_structures.txt', 'w') as f:
	f.write(str(top_structures))

In [11]:
# read top_structures from file
with open(f'top_30_structures.txt', 'r') as f:
	top_structures = ast.literal_eval(f.read())
top_structures

{'KRAS': ['6P0Z',
  '8ONV',
  '8AZZ',
  '4QL3',
  '8AZX',
  '8B00',
  '8AZV',
  '8AZY',
  '8B78',
  '8AFB',
  '4TQA',
  '4LDJ',
  '6TAN',
  '7O70',
  '8EDY',
  '8EER',
  '4Q03',
  '7R0N',
  '8EBZ',
  '8TXH',
  '4OBE',
  '6QUW',
  '5XCO',
  '7RT1',
  '7T47',
  '4LUC',
  '4Q01',
  '6B0V',
  '7RT5',
  '7RPZ'],
 'EGFR': ['8A27',
  '8A2D',
  '5UG9',
  '5HG8',
  '8A2A',
  '5UG8',
  '3POZ',
  '6TFV',
  '6TG0',
  '3VRP',
  '5HG5',
  '5CNO',
  '5UGC',
  '3G5Y',
  '5U8L',
  '6TG1',
  '7SI1',
  '8A2B',
  '3W33',
  '6TFY',
  '4I22',
  '6WXN',
  '6V66',
  '3P0Y',
  '3W32',
  '4I24',
  '5GNK',
  '6TFZ',
  '5UGA',
  '7JXQ'],
 'TP53': ['3D06',
  '5MHC',
  '6GGC',
  '6SHZ',
  '4MZI',
  '6GGE',
  '3LW1',
  '5O1E',
  '6RL3',
  '8E7A',
  '5O1C',
  '5O1H',
  '6GGB',
  '6GGF',
  '7B4N',
  '3ZME',
  '5AOK',
  '5G4N',
  '5O1G',
  '6V4F',
  '5AB9',
  '5O1D',
  '8A92',
  '5G4M',
  '5O1F',
  '7B4H',
  '1YC5',
  '3D08',
  '5A7B',
  '5O1I'],
 'HRAS': ['2CE2',
  '2EVW',
  '2CLD',
  '2CL6',
  '2CL7',
  '5WDQ',
  '1C

In [5]:
from stat import S_ISDIR

def download_directory(sftp, remote_dir, local_dir):
    for item in sftp.listdir_attr(remote_dir):
        remote_path = remote_dir + '/' + item.filename
        local_path = os.path.join(local_dir, item.filename)
        if S_ISDIR(item.st_mode):
            os.makedirs(local_path, exist_ok=True)
            download_directory(sftp, remote_path, local_path)
        else:
            sftp.get(remote_path, local_path)

# get all chains, uniprot id, and start and end in pdb file
def get_dbref_data(pdb_id):
    with open(f'pdb_files/{pdb_id}.pdb', 'r') as f:
        pdb_text = f.read()
        data = []
        for line in pdb_text.split('\n'):
            # contents of a DBREF line:  https://www.wwpdb.org/documentation/file-format-content/format33/sect3.html
            # only reading relevant contents    
            if line.startswith('DBREF') and line[26:32].strip()=="UNP": # if database is UNP 
                data.append({
                    'chain': line[12],
                    'uniprot': line[33:41].strip(),
                    'start': int(line[14:18]),
                    'end': int(line[20:24])
                })  
    return data

get_dbref_data('2F1X')

[{'chain': 'A', 'uniprot': 'Q93009', 'start': 53, 'end': 200},
 {'chain': 'A', 'uniprot': 'P04637', 'start': 201, 'end': 209},
 {'chain': 'B', 'uniprot': 'Q93009', 'start': 53, 'end': 200},
 {'chain': 'B', 'uniprot': 'P04637', 'start': 201, 'end': 209}]

In [5]:
def parse_dbref_line(line):
    # contents of a DBREF line:  https://www.wwpdb.org/documentation/file-format-content/format33/sect3.html
    # only reading relevant contents
    chain_id = line[12]               # Chain identifier
    seq_begin = int(line[14:18])      # Initial sequence number of the PDB sequence segment
    seq_end = int(line[20:24])        # Ending sequence number of the PDB sequence segment
    database = line[26:32].strip()    # Sequence database name
    db_accession = line[33:41].strip()# Sequence database accession code
    
    return {
        "chain_id": chain_id,
        "seq_begin": seq_begin,
        "seq_end": seq_end,
        "database": database,
        "db_accession": db_accession,
    }

# Example usage:
line = "DBREF  6LHD A    2   149L UNP    Q07817   B2CL1_HUMAN      2    201"
parsed_line = parse_dbref_line(line)
print(parsed_line)

{'chain_id': 'A', 'seq_begin': 2, 'seq_end': 149, 'database': 'UNP', 'db_accession': 'Q07817'}


In [6]:
def calculate_sbna_and_download(client, sftp, pdb_id, base_path, chains=None):
    # get available chains
    if not chains: # chain not specified
        try:
            chains = get_dbref_data(pdb_id)
            chains = [chain['chain'] for chain in chains]
        except Exception as e:
            print(f"Failed to get chains for {pdb_id} with error {e}")
            return
    
    sftp.mkdir(f'{base_path}/{pdb_id}')
    for chain in chains: 
        local_dir = f"sbna_results/{pdb_id}/{chain}"
        if os.path.exists(f"{local_dir}/{pdb_id}_monomer/FinalSum"): # skip if analysis already done and downloaded
            continue
        try:
            print(f"\nProcessing {pdb_id} with chain {chain}...")

            # remove the directory if already exists and create new one
            client.exec_command(f"rm -r {base_path}/{pdb_id}/{pdb_id}_monomer")
            client.exec_command(f"rm -r {base_path}/{pdb_id}/{pdb_id}_multimer") 
            
            # upload pdb file to the remote directory
            sftp.put(f"pdb_files/{pdb_id}.pdb", f'{base_path}/{pdb_id}/{pdb_id}.pdb') 
            
            # copy the sbna.sh file to the subdirectory
            client.exec_command(f"cp {base_path}/sbna.sh {base_path}/{pdb_id}/sbna.sh") 

            # run the sbna.sh script
            _,stdout,stderr=client.exec_command(f'cd {base_path}/{pdb_id}; sh sbna.sh {pdb_id}.pdb "Chain {chain}"') 
            print(stdout.read().decode())
            
            # check if Final_sum file exists
            print(stderr.read().decode('utf-8'))
            _,stdout,stderr=client.exec_command(f"ls {base_path}/{pdb_id}/{pdb_id}_monomer/FinalSum")
            # print error message if no output

            # download the sbna results
            if not stdout.read().decode():
                print(f"Final_sum file not found for {pdb_id}")
                # client.exec_command(f"rm -rf {base_path}/{pdb_id}") 
                continue
            
            # download the pdb_id directory
            os.makedirs(local_dir, exist_ok=True) # create the directory if it doesn't exist
            download_directory(sftp, f"{base_path}/{pdb_id}", local_dir)
            print(f"SBNA results for {pdb_id} succesfully downloaded")
        
        except Exception:
            # print full error message
            print(f"Failed to process {pdb_id} with error:")
            print(traceback.print_exc())

            # remove directory
            client.exec_command(f"rm -r {base_path}/{pdb_id}/{pdb_id}_monomer")
            client.exec_command(f"rm -r {base_path}/{pdb_id}/{pdb_id}_multimer")
            continue
    
    client.exec_command(f"rm -rf {base_path}/{pdb_id}") # remove the pdb_id remote directory
    return
    

In [8]:
host = "m3-dtn.massive.org.au"
username = "yliy0004"
with open("pw.txt", "r") as f:
    password = f.read()
base_path = 'ym65_scratch/yliy0004/NetworkAnalysis'

client = paramiko.client.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(host, username=username, password=password)

# check if R module is loaded (uses .bashrc - not the best solution because STRUDEL cannot be used, make sure to correct this if using STRUDEL)
stdin,stdout,stderr=client.exec_command('R --version') 
print(stdout.read().decode())
print(stderr.read().decode())

sftp_client=client.open_sftp()

# ==================== SBNA ====================
# ## multiple structures by gene
# for gene_id in top_structures:
#     for pdb_id in top_structures[gene_id]:

# ## or certain genes
# # for gene in ['HRAS']:
# #     count = 0
#     # tmp = ast.literal_eval(cgc_data.loc[cgc_data['Gene Symbol'] == gene, 'PDB Structures'].values[0])
#     # for pdb_id in tmp:
#     # for pdb_id in top_structures[gene]:
#         try:
#             resolution = get_resolution(pdb_id)
#             if resolution and float(resolution) <= 3.0:
#                 calculate_sbna_and_download(client, sftp_client, pdb_id, base_path)
#         except Exception as e:
#             print(f"Failed to process {pdb_id} with error:")
#             print(traceback.print_exc())
#             continue

## or simply a few pdbs
for pdb_id in ["1D5R", "1JM7", "1T29", "4FMQ", "4NIF"]:
    try:
        calculate_sbna_and_download(client, sftp_client, pdb_id, base_path)
    except Exception as e:
        print(f"Failed to process {pdb_id} with error:")
        print(traceback.print_exc())
        continue

## or one pdb
# calculate_sbna_and_download(client, sftp_client, "1TSR", base_path, chains=['C'])

# close the connection
client.close()
sftp_client.close()

In [13]:
client.close()
sftp_client.close()

# if you want to add more results to final_data
(after having done more sbna iterations)
* reads the pdb structures related to each gene in `cgc_data`
* read the current `final_data.csv`
* appends the new sbna_results to `final_data`

In [8]:
cgc_data = pd.read_csv('Census_all_with_pdb.csv')

In [25]:
cgc_data

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),...,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,PDB Structures,n_structures
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,10q11.23,yes,NaN,melanoma,...,E,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG...",['2CPD'],1
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,10p12.1,yes,NaN,AML,...,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00...",['7LXE'],1
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,9q34.12,yes,NaN,"CML, ALL, T-ALL",...,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007...","['1AB2', '1AWO', '1BBZ', '1JU5', '1OPL', '1ZZP...",81
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,1q25.2,yes,NaN,AML,...,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"ABLL,ARG,CCDS30947.1,ENSG00000143322.19,NM_007...","['2ECD', '2KK1', '2XYN', '3GVU', '3HMI', '3ULR...",9
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,2q37.3,yes,NaN,lipoma,...,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"CCDS2516.1,CMKOR1,CXCR7,ENSG00000144476.5,GPR1...","['6K3F', '7SK3', '7SK4', '7SK5', '7SK6', '7SK7...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,ZNF429,zinc finger protein 429,353088.0,19:21505564-21538078,2,NaN,19p12,yes,NaN,GBM,...,O,NaN,NaN,Mis,NaN,NaN,NaN,"CCDS42537.1,ENSG00000197013.9,NM_001001415.2,N...",NaN,0
739,ZNF479,zinc finger protein 479,90827.0,7:57119614-57139864,2,NaN,7p11.2,yes,NaN,"lung cancer, bladder carcinoma, prostate carci...",...,E,NaN,NaN,Mis,NaN,NaN,NaN,"CCDS43590.1,ENSG00000185177.12,KR19,NM_033273....",NaN,0
740,ZNF521,zinc finger protein 521,25925.0,18:25061926-25352152,1,NaN,18q11.2,yes,NaN,ALL,...,L,Dom,"oncogene, fusion",T,PAX5,NaN,NaN,"CCDS32806.1,EHZF,ENSG00000198795.10,Evi3,NM_01...",NaN,0
741,ZNRF3,zinc and ring finger 3,84133.0,22:28883592-29057487,2,NaN,22q12.1,yes,NaN,"colorectal cancer, adrenocortical carcinoma, g...",...,E,NaN,TSG,"N, F, Mis",NaN,NaN,NaN,"BK747E2.3,CCDS56225.1,ENSG00000183579.15,FLJ22...",NaN,0


In [6]:
def get_uniprot_id(pdb_id, chain):
    # read pdb file 
    with open(f'pdb_files/{pdb_id}.pdb', 'r') as f:
        pdb_text = f.read()
        for line in pdb_text.split('\n'):
            vals = line.split()
            if line.startswith('DBREF') and chain in vals[2] and 'UNP' in vals:
                return vals[6] # uniprot id is the 6th value


def fetch_sequence(uniprot_id):
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
    response = requests.get(url)
    if response.ok:
        # Extract the sequence from the FASTA format
        lines = response.text.split('\n')
        sequence = ''.join(lines[1:])  # Skip the header line
        return sequence
    else:
        print("Failed to fetch sequence")
        return None


def get_gene_name(uniprot_id):
    response = requests.get(f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta")
    response = response.text
    match = re.search(r'GN=([^\s]+)', response)
    if match:
        return match.group(1)
    else:
        return None
    
    
def read_finalsum(pdb_id, chain):
    if os.path.exists(f'sbna_results/{pdb_id}/{chain}/{pdb_id}_monomer/FinalSum'):
        with open(f'sbna_results/{pdb_id}/{chain}/{pdb_id}_monomer/FinalSum', 'r') as f:
            # convert final_sum to dataframe
            final_sum = f.readlines()
            final_sum = [i.split() for i in final_sum]
            rows = []
            for row in final_sum:
                # get letters only from row[0]
                aa_3 = ''.join(filter(str.isalpha, row[0]))
                if row[1] == 'NA' or len(aa_3) != 3: 
                    # skip if residue number is NA or residue code is not 3 letters
                    continue
                
                aa_1 = seq1(aa_3) # convert to 1 letter code
                res_num = ''.join(filter(str.isnumeric, row[0]))
                score = float(row[1])
                # print(f"{aa_3} {aa_1} {res_num} {score}")
                rows.append([aa_3, aa_1, int(res_num), score])
            final_sum = pd.DataFrame(rows, columns=['res', 'res_code', 'num', 'score'])
        return final_sum
    else:
        print(f"FinalSum file not found for {pdb_id}")
        return None

In [7]:
tmp_genes = ["KRAS", "EGFR", "TP53", "HRAS"]
# tmp_genes = ["KRAS", "TP53", "HRAS"]
# tmp_genes = ["EGFR"]

In [ ]:
final_data = []

In [10]:
final_data = list(final_data.values)

In [11]:
filtered_data = cgc_data[cgc_data['PDB Structures'].notna()] # pdb_structures not nan
filtered_data = filtered_data[filtered_data['Gene Symbol'].isin(tmp_genes)]

for i, row in filtered_data.iterrows():
    gene_symbol = row['Gene Symbol']
    pdb_structures = row['PDB Structures']
    for pdb_id in ast.literal_eval(pdb_structures):
        if not os.path.exists(f'pdb_files/{pdb_id}.pdb'):
            continue
        else:
            data = get_dbref_data(pdb_id)
            resolution = get_resolution(pdb_id)
            for i in data:
                chain = i['chain']

                # check if finalsum exists or if data is already in final_data
                if not os.path.exists(f'sbna_results/{pdb_id}/{chain}/{pdb_id}_monomer/FinalSum') or any([pdb_id == i[1] and chain == i[2] for i in final_data]):
                    continue    
                      
                uniprot_id = i['uniprot']
                assoc_gene = get_gene_name(uniprot_id)
                if assoc_gene != gene_symbol:
                    # sometimes protein chain extends multiple genes, skip if not the same
                    # in future can link if gene is in cgc_data/tcga_data
                    continue

                # read finalSum file and add to table
                final_sum = read_finalsum(pdb_id, chain)

                # check if seqeunce of final sum matches uniprot
                uniprot_seq = fetch_sequence(uniprot_id)
                
                for _, row in final_sum.iterrows():
                    num = row['num']
                    start = i['start']
                    end = i['end']
                    outside_range = False
                    residue_match = False
                    try:
                        # get residue from uniprot sequence
                        uniprot_res = uniprot_seq[num-1]
                    except:
                        uniprot_res = None

                    if num < start or num > end: # check if residue number is within range
                        outside_range = True
                        # print(f"Residue {num} not in range {start} - {end}")
                    if row['res_code'] == uniprot_res: # check if residue code matches
                        residue_match = True

                    final_data.append([gene_symbol, pdb_id, chain, uniprot_id, assoc_gene, resolution, num, row['res_code'], uniprot_res, row['score'], outside_range, residue_match])


In [12]:
final_data = pd.DataFrame(final_data, columns=['gene_symbol', 'pdb_id', 'chain', 'uniprot_id', 'assoc_gene', 'resolution', 'res_num', 'pdb_res', 'uniprot_res', 'network_score', 'outside_range', 'residue_match'])
final_data

,gene_symbol,pdb_id,chain,uniprot_id,assoc_gene,resolution,res_num,pdb_res,uniprot_res,network_score,outside_range,residue_match
0,EGFR,3POZ,A,P00533,EGFR,1.5,701,Q,Q,-3.791355,False,True
1,EGFR,3POZ,A,P00533,EGFR,1.5,702,A,A,-2.634055,False,True
2,EGFR,3POZ,A,P00533,EGFR,1.5,703,L,L,-1.231055,False,True
3,EGFR,3POZ,A,P00533,EGFR,1.5,704,L,L,0.452193,False,True
4,EGFR,3POZ,A,P00533,EGFR,1.5,705,R,R,-1.086163,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
52394,TP53,8A92,B,P04637,TP53,1.37,288,N,N,-1.791900,False,True
52395,TP53,8A92,B,P04637,TP53,1.37,289,L,L,-1.017026,False,True
52396,TP53,8A92,B,P04637,TP53,1.37,290,R,R,-1.560145,False,True
52397,TP53,8A92,B,P04637,TP53,1.37,291,K,K,-2.160481,False,True


In [13]:
final_data = final_data.sort_values(["assoc_gene", "pdb_id", "chain", "uniprot_id"], ascending=True)
final_data.to_csv('final_data.csv', index=False)

In [9]:
final_data = pd.read_csv('final_data.csv')

In [14]:
# count number of structures for each gene
final_data.groupby('assoc_gene')['pdb_id'].nunique()


assoc_gene
EGFR     28
HRAS     30
KRAS     31
TP53    109
Name: pdb_id, dtype: int64